In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0


train_dir = '/kaggle/input/cutmix-17march/cutmix_new/train'
test_dir = '/kaggle/input/cutmix-17march/cutmix_new/test'

img_height = 224
img_width = 224
batch_size = 32

preprocess_input = tf.keras.applications.efficientnet.preprocess_input

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

efficientnet_base = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

for layer in efficientnet_base.layers:
    layer.trainable = False

x = layers.GlobalAveragePooling2D()(efficientnet_base.output)
x = layers.Dense(256, activation='relu')(x)
output = layers.Dense(1, activation='sigmoid')(x)

model = models.Model(inputs=efficientnet_base.input, outputs=output)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_generator,
                    steps_per_epoch=None,  # Set to None to run until the generator stops
                    epochs=10,
                    validation_data=test_generator,
                    validation_steps=None)  # Set to None to run until the generator stops

loss, accuracy = model.evaluate(test_generator)
print("Test Accuracy:", accuracy)


2024-03-17 12:05:16.530310: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-17 12:05:16.530425: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-17 12:05:16.676411: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 1294 images belonging to 2 classes.
Found 856 images belonging to 2 classes.
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


41/41 ━━━━━━━━━━━━━━━━━━━━ 215s 4s/step - accuracy: 0.7000 - loss: 0.5619 - val_accuracy: 0.8727 - val_loss: 0.3138
Epoch 2/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 233s 4s/step - accuracy: 0.8663 - loss: 0.3131 - val_accuracy: 0.8890 - val_loss: 0.2841
Epoch 3/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 206s 4s/step - accuracy: 0.8976 - loss: 0.2599 - val_accuracy: 0.8925 - val_loss: 0.2750
Epoch 4/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 195s 4s/step - accuracy: 0.9315 - loss: 0.2056 - val_accuracy: 0.9100 - val_loss: 0.2450
Epoch 5/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 201s 4s/step - accuracy: 0.9601 - loss: 0.1355 - val_accuracy: 0.9136 - val_loss: 0.2360
Epoch 6/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 200s 4s/step - accuracy: 0.9683 - loss: 0.1234 - val_accuracy: 0.8645 - val_loss: 0.3682
Epoch 7/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 203s 4s/step - accuracy: 0.9625 - loss: 0.1291 - val_accuracy: 0.8808 - val_loss: 0.3627
Epoch 8/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 202s 4s/step - accuracy: 0.9666 - loss: 0.1056 - val_accuracy: 0.9042 - val_loss: 0.

In [7]:
image_path = '/kaggle/input/monalisa-predict-cutmix/monalisa.jpeg'

img = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_height, img_width))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

prediction = model.predict(img_array)

if prediction < 0.5:
    print("AI generated Image")
else:
    print("Original Image")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Original Image
